In [26]:
import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import google.generativeai as genai
from huggingface_hub import login
import gradio as gr
from IPython.display import Markdown, display
import ollama

In [27]:
load_dotenv()

hf_token = os.getenv("HUGGINGFACE_TOKEN")
login(hf_token)

## Importing the keys

In [28]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
claude_api_key = os.getenv('CLAUDE_API_KEY')
MODEL_OPENAI = 'gpt-4o-mini'
MODEL_GEMINI = 'gemini-1.5-flash'
MODEL_DEEPSEEK = "deepseek/deepseek-chat"
MODEL_LLAMA = 'llama3.2'
MODEL_CLAUDE_SONNET = "anthropic/claude-3.7-sonnet"
openai = OpenAI()

genai.configure(api_key=gemini_api_key)
gemini = genai.GenerativeModel(MODEL_GEMINI)

deepseek = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=deepseek_api_key,
)

claude = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key=claude_api_key,
)

print(openai_api_key[:7])
print(gemini_api_key[:3])
print(deepseek_api_key[:4])
print(claude_api_key[:7])

sk-proj
AIz
sk-o
sk-or-v


## System and User Prompts 

In [29]:
system_prompt = """
You are a business analyst at a company that provides security systems and smart home solutions. 
Read the following chat transcripts between customers and support agents. 
Your task is to extract general insights such as:
- recurring problems,
- product feedback,
- service issues,
- and any trends that could help ADT improve customer experience or business strategy.
Your summary should be factual, concise, and suitable for executives.
"""

user_prompt = """
Chat 1:
Customer: My door sensor keeps showing low battery even after I replace it.
Agent: I understand, let me check if it’s a known issue with the firmware...

Chat 2:
Customer: My smart camera goes offline every night.
Agent: That sounds like a router scheduling setting, let’s check it...


Please summarize all the insights and patterns you can infer from the above chats.
"""


## OpenAI's response

In [30]:
response = openai.chat.completions.create(
    model=MODEL_OPENAI,
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.3
)

result = response.choices[0].message.content.strip()
display(Markdown(result))



**Insights and Patterns from Customer Support Chats:**

1. **Recurring Problems:**
   - **Door Sensor Battery Issues:** Multiple customers are experiencing low battery notifications for door sensors even after battery replacement, indicating a potential firmware or hardware issue.
   - **Smart Camera Connectivity:** Customers report smart cameras going offline regularly, suggesting possible connectivity issues related to router settings or device compatibility.

2. **Product Feedback:**
   - There is a need for enhanced reliability in battery monitoring for door sensors and improved connectivity for smart cameras, as these issues could lead to customer dissatisfaction and reduced trust in product performance.

3. **Service Issues:**
   - The support agents are required to troubleshoot potential underlying issues, such as firmware problems and router settings, which may indicate a gap in product documentation or user guidance.

4. **Trends for Improvement:**
   - **Firmware Updates:** Investigating and addressing firmware issues for door sensors could enhance product reliability.
   - **User Education:** Providing clearer instructions on router settings and device management could help customers avoid connectivity issues with smart cameras.
   - **Proactive Support:** Implementing a system to notify customers of known issues and solutions could improve the overall customer experience and reduce the volume of support inquiries.

These insights suggest areas for product development, customer education, and service enhancement to improve overall customer satisfaction and loyalty.

## Gemini's response

In [31]:


response = gemini.generate_content(system_prompt + "\n" + user_prompt)
display(Markdown(response.text))

Analysis of Customer Chat Transcripts:

**Recurring Problems:**

* **Faulty/Inconsistent Sensor Battery Readings:**  At least one customer reports inaccurate low battery readings on a door sensor despite battery replacement, suggesting a potential hardware or firmware issue.

* **Smart Camera Connectivity Issues:**  One customer experiences nightly offline periods with their smart camera, pointing to potential network configuration problems (router settings) or intermittent camera failures.


**Product Feedback (Implicit):**

* **Door Sensor Reliability:** The battery issue highlights concerns about the reliability and accuracy of the door sensor's reporting mechanism.
* **Smart Camera Stability:** The reported connectivity issue suggests a need for improved camera network stability and possibly more robust error handling.


**Service Issues:**

* No direct service issues are evident in these short transcripts, but the need for troubleshooting suggests potential areas for improvement in product design or user documentation (e.g., clearer instructions on router settings affecting smart devices).


**Trends and Recommendations:**

* **Investigate Door Sensor Firmware/Hardware:** A deeper investigation into the root cause of the inaccurate battery readings in Chat 1 is crucial. This could involve firmware updates, hardware revisions, or improved battery detection algorithms.

* **Improve Smart Camera Network Compatibility:** The camera's nightly offline periods indicate a need for improved documentation on network configuration, or potential design changes to enhance the camera's robustness to common router settings or network fluctuations.

* **Proactive Communication:** Consider proactively informing customers about known issues (e.g., via app notifications or email) to prevent frustration.

* **Enhance User Documentation:** Clearer and more comprehensive user manuals and in-app instructions are needed to address common technical hurdles.  This should particularly address network configuration for smart devices.

The limited data prevents definitive conclusions, but these initial findings suggest areas warranting further investigation to improve both product quality and customer satisfaction.  More data is needed for more comprehensive analysis.


## Llama 3.2

In [34]:
messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

response = ollama.chat(model=MODEL_LLAMA, messages=messages)
reply = response['message']['content']
display(Markdown(reply))

RemoteProtocolError: Server disconnected without sending a response.

## Deepseek

In [11]:
messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ]

response = deepseek.chat.completions.create(
    model=MODEL_DEEPSEEK,
    messages=messages
)
display(Markdown(response.choices[0].message.content))

**Insights from Chat Transcripts:**

1. **Recurring Problems:**
   - **Battery Issues:** Customers are experiencing persistent low battery notifications with ADT door sensors, even after replacing the batteries. This suggests a potential firmware or hardware issue.
   - **Device Connectivity:** Smart cameras are frequently going offline, particularly at night, which may be linked to router scheduling settings or Wi-Fi stability.

2. **Product Feedback:**
   - **Door Sensor Reliability:** The consistent low battery issue with door sensors indicates a need for either firmware updates or hardware improvements to enhance reliability.
   - **Smart Camera Connectivity:** The smart camera’s offline problem highlights a possible need for better integration with common router settings or improved Wi-Fi compatibility.

3. **Service Issues:**
   - **Technical Support Response:** Agents are identifying potential causes (e.g., firmware checks, router settings), but recurring issues suggest a need for more proactive solutions or clearer troubleshooting guides for customers.

4. **Trends:**
   - **Firmware/Software Concerns:** Firmware appears to be a common factor in device malfunctions, warranting a review of device software updates.
   - **Network Integration:** Connectivity issues point to a need for better compatibility with home network setups, including routers and Wi-Fi systems.

**Key Recommendations:**
- Investigate and address firmware issues with door sensors to resolve low battery notifications.
- Enhance smart camera compatibility with router scheduling and Wi-Fi systems to prevent offline incidents.
- Develop proactive troubleshooting support materials for common issues to improve customer self-service.
- Consider a review of firmware and software updates across devices to ensure optimal performance and reliability.

## Gradio (take inputs for system prompt and user prompt from users)

In [21]:
import gradio as gr
from IPython.display import Markdown, display

def generate_summary(llm_choice, system_prompt, user_prompt):
    if llm_choice == "OPENAI":
        response = openai.chat.completions.create(
            model=MODEL_OPENAI,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3
        )
        return response.choices[0].message.content.strip()
        
    elif llm_choice == "CLAUDE":
        try:
            response = claude.messages.create(
                model="anthropic/claude-3.7-sonnet",
                messages=[
                    {"role": "user", "content": f"{system_prompt}\n\n{user_prompt}"}
                ],
                temperature=0.3,
                extra_headers={
                    "HTTP-Referer": "https://example.com",  # required by OpenRouter
                    "X-Title": "ADT Transcript Insight Generator"
                }
            )
            return response.message.content[0].text.strip()
        except Exception as e:
            return f"Claude failed: {str(e)}"


        
        

    elif llm_choice == "GEMINI":
        response = gemini.generate_content(system_prompt + "\n" + user_prompt)
        return response.text

    elif llm_choice == "DEEPSEEK":
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        response = deepseek.chat.completions.create(
            model=MODEL_DEEPSEEK,
            messages=messages
        )
        return response.choices[0].message.content

    else:
        return "Invalid model selection."

# Create the interface
gr.Interface(
    fn=generate_summary,
    inputs=[
        gr.Dropdown(choices=["OPENAI", "GEMINI", "DEEPSEEK", "CLAUDE"], label="Select LLM"),
        gr.Textbox(lines=6, label="System Prompt"),
        gr.Textbox(lines=12, label="User Prompt"),
    ],
    outputs=gr.Textbox(label="LLM Response"),
    title="ADT Transcript Insight Generator"
).launch()


* Running on local URL:  http://127.0.0.1:7871

To create a public link, set `share=True` in `launch()`.


## Gradio (pre-defined system and user prompt)

In [13]:
import gradio as gr

def generate_summary(llm_choice):
    yield "⏳ Generating response from " + llm_choice + "..."
    
    if llm_choice == "OPENAI":
        response = openai.chat.completions.create(
            model=MODEL_OPENAI,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.3
        )
        result = response.choices[0].message.content.strip()

    elif llm_choice == "CLAUDE SONNET 3.7":
        response = claude.chat.completetions.create(
            model=MODEL_CLAUDE_SONNET,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature = 0.3
        )

        result = response.choices[0].message.content.strip()

    elif llm_choice == "GEMINI":
        response = gemini.generate_content(system_prompt + "\n" + user_prompt)
        result = response.text

    elif llm_choice == "DEEPSEEK":
        messages = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ]
        response = deepseek.chat.completions.create(
            model=MODEL_DEEPSEEK,
            messages=messages
        )
        result = response.choices[0].message.content
    elif llm_choice == 'LLAMA':
        messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ]
        response = ollama.chat(model=MODEL_LLAMA, messages=messages)
        result = response['message']['content']
        


    else:
        result = "❌ Invalid model selection."

    yield result

# Interface with streaming output
gr.Interface(
    fn=generate_summary,
    inputs=gr.Dropdown(choices=["OPENAI", "GEMINI", "DEEPSEEK", "LLAMA", "CLAUDE SONNET 3.7"], label="Select LLM"),
    outputs=gr.Markdown(label="LLM Response"),
    title="ADT Transcript Insight Generator",
    live=True
).launch()


* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/harshakparmar/Desktop/LLM-based_projects/venv/lib/python3.11/site-packages/gradio/queueing.py", line 715, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harshakparmar/Desktop/LLM-based_projects/venv/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harshakparmar/Desktop/LLM-based_projects/venv/lib/python3.11/site-packages/gradio/blocks.py", line 2137, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harshakparmar/Desktop/LLM-based_projects/venv/lib/python3.11/site-packages/gradio/blocks.py", line 1675, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harshak